In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split , KFold
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

from collections import Counter

In [5]:
iris_dataset=pd.read_excel('Iris.xls')

In [6]:
iris_dataset

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


In [8]:
x= iris_dataset.iloc[:, :-1]
y= iris_dataset.iloc[:, -1]

In [9]:
x.head()

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length
0,1,0.2,1.4,3.5,5.1
1,1,0.2,1.4,3.0,4.9
2,1,0.2,1.3,3.2,4.7
3,1,0.2,1.5,3.1,4.6
4,1,0.2,1.4,3.6,5.0


In [10]:
y.head()

0     Setosa
1     Setosa
2     Setosa
3     Setosa
4     Setosa
Name: Species_name, dtype: object

In [11]:
# split the data into train and test sets
x_train, x_test, y_train, y_test= train_test_split(x, y,
                                                   test_size= 0.2,
                                                   shuffle= True, #shuffle the data to avoid bias
                                                   random_state= 0)
x_train= np.asarray(x_train)
y_train= np.asarray(y_train)

x_test= np.asarray(x_test)
y_test= np.asarray(y_test)

In [12]:
print(f'training set size: {x_train.shape[0]} samples \ntest set size: {x_test.shape[0]} samples')

training set size: 120 samples 
test set size: 30 samples


#Normalize the dataset

In [13]:
scaler= Normalizer().fit(x_train)
normalized_x_train= scaler.transform(x_train)
normalized_x_test= scaler.transform(x_test) 

In [18]:
print('x train before Normalization')
print(x_train[0:5])
print('\nx train after Normalization')
print(normalized_x_train[0:5])

x train before Normalization
[[3.  1.8 5.5 3.1 6.4]
 [2.  1.5 4.5 3.  5.4]
 [1.  0.2 1.5 3.5 5.2]
 [3.  1.8 4.9 3.  6.1]
 [3.  2.2 5.6 2.8 6.4]]

x train after Normalization
[[0.31098521 0.18659112 0.57013955 0.32135138 0.66343511]
 [0.24872082 0.18654062 0.55962185 0.37308123 0.67154622]
 [0.15324883 0.03064977 0.22987325 0.53637091 0.79689392]
 [0.33036923 0.19822154 0.53960307 0.33036923 0.67175077]
 [0.30942637 0.22691267 0.5775959  0.28879795 0.6601096 ]]


#Visualize the Dataset before and after Normalization

#KNN Step 1 (Euclidean Distance)

In [19]:
def distance_ecu(x_train, x_test_point):
  """
  Input:
    - x_train: corresponding to the training data
    - x_test_point: corresponding to the test point

  Output:
    -distances: The distances between the test point and each point in the training data.

  """
  distances= []  ## create empty list called distances
  for row in range(len(x_train)): ## Loop over the rows of x_train
      current_train_point= x_train[row] #Get them point by point
      current_distance= 0 ## initialize the distance by zero

      for col in range(len(current_train_point)): ## Loop over the columns of the row
          
          current_distance += (current_train_point[col] - x_test_point[col]) **2
          ## Or current_distance = current_distance + (x_train[i] - x_test_point[i])**2
      current_distance= np.sqrt(current_distance)

      distances.append(current_distance) ## Append the distances

  # Store distances in a dataframe
  distances= pd.DataFrame(data=distances,columns=['dist'])
  return distances


#KNN Step 2 (Find the nearest neighbors)

In [20]:
def nearest_neighbors(distance_point, K):
    """
    Input:
        -distance_point: the distances between the test point and each point in the training data.
        -K             : the number of neighbors

    Output:
        -df_nearest: the nearest K neighbors between the test point and the training data.

    """

    # Sort values using the sort_values function
    df_nearest= distance_point.sort_values(by=['dist'], axis=0)

    ## Take only the first K neighbors
    df_nearest= df_nearest[:K]
    return df_nearest


##KNN Step 3 (Classify the point based on a majority vote)

In [21]:
def voting(df_nearest, y_train):
    """
    Input:
        -df_nearest: dataframe contains the nearest K neighbors between the full training dataset and the test point.
        -y_train: the labels of the training dataset.

    Output:
        -y_pred: the prediction based on Majority Voting

    """

    ## Use the Counter Object to get the labels with K nearest neighbors.
    counter_vote= Counter(y_train[df_nearest.index])

    y_pred= counter_vote.most_common()[0][0]   # Majority Voting

    return y_pred
#KNN Full Algorithm: Putting Everything Together

def KNN_from_scratch(x_train, y_train, x_test, K):

    """
    Input:
    -x_train: the full training dataset
    -y_train: the labels of the training dataset
    -x_test: the full test dataset
    -K: the number of neighbors

    Output:
    -y_pred: the prediction for the whole test set based on Majority Voting.

    """

    y_pred=[]

    ## Loop over all the test set and perform the three steps
    for x_test_point in x_test:
      distance_point  = distance_ecu(x_train, x_test_point)  ## Step 1
      df_nearest_point= nearest_neighbors(distance_point, K)  ## Step 2
      y_pred_point    = voting(df_nearest_point, y_train) ## Step 3
      y_pred.append(y_pred_point)

    return y_pred  


#Test the KNN Algorithm on the test dataset

In [22]:
K=3
y_pred_scratch= KNN_from_scratch(normalized_x_train, y_train, normalized_x_test, K)
print(y_pred_scratch)

[' Verginica', ' Versicolor', ' Setosa', ' Verginica', ' Setosa', ' Verginica', ' Setosa', ' Versicolor', ' Versicolor', ' Versicolor', ' Verginica', ' Versicolor', ' Versicolor', ' Versicolor', ' Versicolor', ' Setosa', ' Versicolor', ' Versicolor', ' Setosa', ' Setosa', ' Verginica', ' Versicolor', ' Setosa', ' Setosa', ' Verginica', ' Setosa', ' Setosa', ' Versicolor', ' Versicolor', ' Setosa']


#Compare our implementation with Sklearn library

In [23]:
knn=KNeighborsClassifier(K)
knn.fit(normalized_x_train, y_train)
y_pred_sklearn= knn.predict(normalized_x_test)
print(y_pred_sklearn)

[' Verginica' ' Versicolor' ' Setosa' ' Verginica' ' Setosa' ' Verginica'
 ' Setosa' ' Versicolor' ' Versicolor' ' Versicolor' ' Verginica'
 ' Versicolor' ' Versicolor' ' Versicolor' ' Versicolor' ' Setosa'
 ' Versicolor' ' Versicolor' ' Setosa' ' Setosa' ' Verginica'
 ' Versicolor' ' Setosa' ' Setosa' ' Verginica' ' Setosa' ' Setosa'
 ' Versicolor' ' Versicolor' ' Setosa']


#Check if the output is exactly the same

In [24]:
print(np.array_equal(y_pred_sklearn, y_pred_scratch))

True


#Calculate the accuracy of both methods

In [25]:
print(f'The accuracy of our implementation is {accuracy_score(y_test, y_pred_scratch)}')
print(f'The accuracy of sklearn implementation is {accuracy_score(y_test, y_pred_sklearn)}')

The accuracy of our implementation is 1.0
The accuracy of sklearn implementation is 1.0


In [26]:
n_splits= 4 ## Choose the number of splits
kf= KFold(n_splits= n_splits) ## Call the K Fold function

accuracy_k= [] ## Keep track of the accuracy for each K
k_values= list(range(1,30,2)) ## Search for the best value of K

for k in k_values: ## Loop over the K values
  accuracy_fold= 0
  for normalized_x_train_fold_idx, normalized_x_valid_fold_idx in  kf.split(normalized_x_train): ## Loop over the splits
      normalized_x_train_fold= normalized_x_train[normalized_x_train_fold_idx] ## fetch the values
      y_train_fold= y_train[normalized_x_train_fold_idx]

      normalized_x_test_fold= normalized_x_train[normalized_x_valid_fold_idx]
      y_valid_fold= y_train[normalized_x_valid_fold_idx]
      y_pred_fold= KNN_from_scratch(normalized_x_train_fold, y_train_fold, normalized_x_test_fold, k)

      accuracy_fold+= accuracy_score (y_pred_fold, y_valid_fold) ## Accumulate the accuracy
  accuracy_fold= accuracy_fold/ n_splits ## Divide by the number of splits
  accuracy_k.append(accuracy_fold)

  

In [27]:
print(f'The accuracy for each K value was {list ( zip (accuracy_k, k_values))}')

The accuracy for each K value was [(0.9916666666666667, 1), (0.9916666666666667, 3), (0.9833333333333334, 5), (0.9833333333333334, 7), (0.9833333333333334, 9), (0.9833333333333334, 11), (0.9833333333333334, 13), (0.9833333333333334, 15), (0.9833333333333334, 17), (0.9833333333333334, 19), (0.9833333333333334, 21), (0.975, 23), (0.975, 25), (0.975, 27), (0.975, 29)]


In [28]:
print(f'Best accuracy was {np.max(accuracy_k)}, which corresponds to a value of K= {k_values[np.argmax(accuracy_k)]}')

Best accuracy was 0.9916666666666667, which corresponds to a value of K= 1
